In [1]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.functions import sha2, concat_ws, col, concat, when, length, lpad, to_date

In [2]:
#function to read later

def control_null (dataframe): 
    for dfcol in dataframe.columns:
        df_null = dataframe.filter(col(dfcol).isNull()).count()
        if df_null > 0:
            print(f"{dfcol} : {df_null} null")
        else:
            print(f"{dfcol} no tiene null")

def create_hash(Datafram):
    value = Datafram.withColumn("hash_id", sha2(concat_ws("||", *Datafram.columns), 256))
    return value

def write_DB (parquet_DF, table):
    try:
        parquet_DF.write.jdbc(
            url=jdbc_url,
            table=table,
            mode="append",
            properties=properties
        )
    except Exception as error:
        logger.error("Error write into DB:" + str(error))


In [3]:

spark = (
    SparkSession.builder
    .appName("Spark")
    .config("spark.driver.extraJavaOptions", r'-Dlog4j.configurationFile=file:/home/illidan/proyecto_desde0/ETL/log4j.properties')\
    .config("spark.jars", "/home/illidan/proyecto_desde0/postgre/jars/postgresql-42.7.4.jar") \
    .getOrCreate()
)

spark.sparkContext.setLogLevel("INFO")
logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)



logger.info("Log de ejemplo guardado en archivo y consola.")

errores_detectados = []

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


03:03:11.358 [Thread-4] INFO  __main__ - Log de ejemplo guardado en archivo y consola.


In [4]:
try:
    #reading the config file
    #geting file path into a dictionary
    with open("/home/illidan/proyecto_desde0/Config_file/Config.Yaml", "r") as file:
        config = yaml.safe_load(file)
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))
    
try:
    read_parquet_airline = config["Parquet_file"]["df_airline"]
    read_parquet_flights = config["Parquet_file"]["df_flights"]
    read_parquet_airports = config["Parquet_file"]["df_airports"]
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))

In [5]:
#BD conection

try:
    jdbc_url = "jdbc:postgresql://localhost:5433/sparkdb_dev"
    properties = {
        "user": "spark", #config['sparkdb_dev']['user'],
        "password": "spark", #config['sparkdb_dev']['password'],
        "driver": "org.postgresql.Driver"
    }
except Exception as error:
    logger.error("Error conection BD:" + str(error))

In [6]:
try:
    df_flights = spark.read.parquet(read_parquet_flights)
    df_airline = spark.read.parquet(read_parquet_airline)
    df_airports = spark.read.parquet(read_parquet_airports)
except Exception as error:
    logger.error("Error read.parquet:" + str(error))

03:03:11.418 [Thread-4] INFO  org.apache.spark.sql.internal.SharedState - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
03:03:11.422 [Thread-4] INFO  org.apache.spark.sql.internal.SharedState - Warehouse path is 'file:/home/illidan/proyecto_desde0/ETL/spark-warehouse'.
03:03:11.446 [Thread-4] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@42d78688{/SQL,null,AVAILABLE,@Spark}
03:03:11.448 [Thread-4] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@1afe699d{/SQL/json,null,AVAILABLE,@Spark}
03:03:11.449 [Thread-4] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@6f0be16a{/SQL/execution,null,AVAILABLE,@Spark}
03:03:11.451 [Thread-4] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@2886a741{/SQL/execution/json,null,AVAILABLE,@Spark}
03:03:11.453 [Threa

03:03:13.819 [Executor task launch worker for task 0.0 in stage 0.0 (TID 0)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 0.0 (TID 0). 3053 bytes result sent to driver
03:03:13.831 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 0.0 (TID 0) in 507 ms on 172.23.57.81 (executor driver) (1/1)
03:03:13.833 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed TaskSet 0.0, whose tasks have all completed, from pool 
03:03:13.838 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ResultStage 0 (parquet at NativeMethodAccessorImpl.java:0) finished in 0.701 s
03:03:13.841 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
03:03:13.842 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Killing all running tasks in stage 0: Stage finish

In [7]:
#combine the year, month and day columns into a single date column
try:
        df_flights = df_flights.withColumn("DATE", 
                                                to_date(
                                                        concat(
                                                                col("YEAR"),
                                                                when(length(col("MONTH")) == 1, lpad(col("MONTH"), 2, "0")).otherwise(col("MONTH")),
                                                                when(length(col("DAY")) == 1, lpad(col("DAY"), 2, "0")).otherwise(col("DAY"))
                                                        ),\
                                                        "yyyyMMdd"
                                                )
                                        )
except Exception as error:
    logger.error("Error al crear la columna DATE:" + str(error))

In [8]:
#Create hash_id to identify the rows when insert into the database
try:
    df_airline = create_hash(df_airline)
    df_airports = create_hash(df_airports)
    df_flights = create_hash(df_flights)
except Exception as error:
    logger.error("Error create_hash:" + str(error))

03:03:15.568 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_1_piece0 on 172.23.57.81:33405 in memory (size: 37.2 KiB, free: 434.4 MiB)
03:03:15.574 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_2_piece0 on 172.23.57.81:33405 in memory (size: 37.2 KiB, free: 434.4 MiB)


In [ ]:
#charge the data into the database

#write_DB(df_airline, "airlines")

DB_airline = spark.read.jdbc(url=jdbc_url, table="airlines", properties=properties).select("IATA_CODE", "hash_id")

#find the rows in df_airline that are not in the database
#iF the id in DF is not in the database, insert it
Airline_to_insert = df_airline.join(DB_airline, df_airline.IATA_CODE == DB_airline.IATA_CODE, "left_anti")
write_DB(Airline_to_insert, "airlines")

#this are the rows thar are in the database by IATA_CODE
#if the Id in DF is in the database, we need to check if the hash_id is the same, if is not, we need to update the row, but if it is the same, we do nothing
rows_same_Id = df_airline.join(DB_airline, df_airline.IATA_CODE == DB_airline.IATA_CODE, "inner").select(df_airline.IATA_CODE, df_airline.hash_id)

rows_to_update = rows_same_Id.join(DB_airline, rows_same_Id.hash_id == DB_airline.hash_id, "left_anti")






15:13:01.311 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: 
15:13:01.311 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: 
15:13:01.459 [Thread-4] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_82 stored as values in memory (estimated size 200.9 KiB, free 427.4 MiB)
15:13:01.507 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_81_piece0 on 172.23.57.81:33405 in memory (size: 22.8 KiB, free: 434.3 MiB)
15:13:01.534 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_76_piece0 on 172.23.57.81:33405 in memory (size: 34.9 KiB, free: 434.3 MiB)
15:13:01.537 [Thread-4] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_82_piece0 stored as bytes in memory (estimated size 34.9 KiB, free 427.7 MiB)
15:13:01.537 [dispatcher-BlockManagerMaster] INFO  org.apache.

In [10]:


df_airline.printSchema()




root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- hash_id: string (nullable = true)

